In [1]:
### Try using a neural network to learn a function just from random space points and the exact value
### See how the error goes down as the runtime, size of network, methodology change

# Solution to Laplace's equation
# u(x,y) = 4.0*(x - 1.0)**2 - 4.0*(y + 0.5)**2
#  in a polar region r = 0.75*cos(theta) + 1.1
#  Compare PDE solver with Direct solver

# High frequency 
# u(x,y) = sin( 4.2*x ) cos( 7.4*y).^2

# Radial function
# u(x,y) = sin( sqrt(x^2 + y^2) )

# High dimensional d=50
# u(x1, ..., x50) =

# Peaks
#  z = 3*(1-x).^2.*exp(-(x.^2) - (y+1).^2) - 10*(x/5 - x.^3 - y.^5).*exp(-x.^2-y.^2) - 1/3*exp(-(x+1).^2 - y.^2) 

# Franke's function
# z = 3/4 * e^(- ( (9x - 2)^2 + (9y -2)^2 )/4 ) + 3/4 * e^( -( (9x-1)^2/49 - (9y+1)/10 )) + 1/2 * e^( -( (9x-7)^2 + (9y-3)^2 )/4 ) - 1/5 * e^( -(  (9x-4)^2 + (9y-7)^2 ))

In [2]:
import torch
import numpy as np
import time
import math
import importlib
import pickle

import DRLPDE.create as create
import DRLPDE.neuralnets as neuralnets
import DRLPDE.train as train
import DRLPDE.solver as solver
import DRLPDE.main as main
import DRLPDE.diagnostics as diagnostics

torch.set_default_dtype(torch.float32)

use_cuda=torch.cuda.is_available()
dev = torch.device("cuda:0" if use_cuda else "cpu")

In [3]:
parameters = 'example1'
solver = {'savemodel':'Test_SeparateOptimizers', 
         'trainingsteps':100,
         'method':{'type':'direct'},
         'optimizer':{'learningrate': 1e-4,
                'beta': (0.9, 0.999),
                'weightdecay': 0}}

In [4]:
# Use GPU if available
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Parameters of the Problem

if parameters:
    param = importlib.import_module("." + parameters, package='examples')
    problem = 'examples/' + parameters + '.py'
else:
    import DRLPDE.parameters as param
    problem = 'default_parameters.py'

# Dimensions of problem
input_dim = [param.x_dim, param.t_dim, param.hyper_dim]

output_dim = param.output_dim

# Intervals
input_range = param.boundingbox + param.t_range + param.hyper_range

# Boolean for each wall type
there_are_walls = any(param.list_of_walls)
there_are_solids = any(param.solid_walls)
there_are_inletoutlets = any(param.inlet_outlet)
there_are_meshes = any(param.mesh)
unsteady = bool(param.t_dim)

# Diagnostics 
collect_error = param.collect_error
if collect_error:
    num_error = param.num_error
    true_fun = param.true_fun

solver_parameters = main.define_solver_parameters(problem, **solver)

# Training parameters
trainingsteps = int(solver_parameters['trainingsteps'])
num = solver_parameters['numpts']
numbatch = solver_parameters['numbatch']
resample_every = solver_parameters['resample_every']
walk = solver_parameters['walk']
importance_sampling = solver_parameters['importance_sampling']
reweight_every = solver_parameters['adaptive_weighting']['reweight_every']
stepsize = solver_parameters['adaptive_weighting']['stepsize']
print_every = round(trainingsteps/10)

In [5]:
### Initialize the Neural Network

# TODO: Choose other neural network architectures from neuralnets.py
if solver_parameters['neuralnetwork'] == 'FeedForward':
    MyNeuralNetwork = neuralnets.FeedForwardNN
elif solver_parameters['neuralnetwork'] == 'Incompressible':
    MyNeuralNetwork = neuralnets.IncompressibleNN

nn_size = solver_parameters['nn_size']

model = MyNeuralNetwork(input_dim, output_dim, **nn_size).to(dev)
    
optimizer1 = torch.optim.Adam(model.parameters(), 
                                lr=solver_parameters['optimizer']['learningrate'], 
                                betas=solver_parameters['optimizer']['beta'], 
                                weight_decay=solver_parameters['optimizer']['weightdecay'])

optimizer2 = torch.optim.Adam(model.parameters(), 
                                lr=solver_parameters['optimizer']['learningrate'], 
                                betas=solver_parameters['optimizer']['beta'], 
                                weight_decay=solver_parameters['optimizer']['weightdecay'])


In [6]:
# Define the Physical domain through its boundaries
Domain = create.SpaceDomain(param.boundingbox, 
                            param.list_of_walls, 
                            param.solid_walls, 
                            param.inlet_outlet, 
                            param.list_of_periodic_ends, 
                            param.mesh)

wall_measure = Domain.wall[0].measure

# Create Interior points and organize into batches
IntPoints = create.InteriorPoints(num, Domain, input_dim, input_range)
IntPoints_batch = torch.utils.data.DataLoader(IntPoints, batch_size=numbatch, shuffle=True)

# Create Dirichlet Wall points and organize into batches
num_wall = create.numBCpoints(num, input_dim, Domain, Domain.wall)
BCPoints = create.BCPoints(num_wall, Domain, input_dim, input_range)
BCPoints_batch = torch.utils.data.DataLoader(BCPoints, batch_size=numbatch, shuffle=True)

### Random points for calculating L2 error through Monte Carlo Integration
ErrorPoints = create.InteriorPoints(num_error, Domain, input_dim, input_range)
ErrorPoints_batch = torch.utils.data.DataLoader(ErrorPoints, batch_size=numbatch, shuffle = True)


In [7]:
#solver_parameters['method']['type']
### Method type
if solver_parameters['method']['type'] == 'autodiff':
    import DRLPDE.autodiff as method
    var_train = {'diffusion': param.diffusion,
                    'forcing': param.forcing,
                    'x_dim': input_dim[0]
                }
    
        ### PDE
    if unsteady:
        if param.pde_type == 'NavierStokes':
            make_target = method.unsteadyNavierStokes
        elif param.pde_type == 'viscousBurgers':
            make_target = method.unsteadyViscousBurgers
    else:
        if param.pde_type == 'NavierStokes':
            make_target = method.steadyNavierStokes
        elif param.pde_type == 'viscousBurgers':
            make_target = method.steadyViscousBurgers
        elif param.pde_type == 'Stokes':
            make_target = method.Laplace
        elif param.pde_type == 'Laplace':
            make_target = method.Laplace

elif solver_parameters['method']['type'] == 'stochastic':
    import DRLPDE.stochastic as method
    var_train = {'diffusion': param.diffusion,
                'forcing': param.forcing, 
                'x_dim': input_dim[0],
                'domain': Domain,
                'dt': solver_parameters['method']['dt'],
                'num_ghost': solver_parameters['method']['num_ghost'], 
                'tol': solver_parameters['method']['tol']
                }

    ### PDE
    if unsteady:
        var_train['ic'] = param.init_con
        if param.pde_type == 'NavierStokes':
            make_target = method.unsteadyNavierStokes
        elif param.pde_type == 'viscousBurgers':
            make_target = method.unsteadyViscousBurgers
        elif param.pde_type == 'Stokes':
            make_target = method.Heat
        elif param.pde_type == 'Heat':
            make_target = method.Heat
    else:
        if param.pde_type == 'NavierStokes':
            make_target = method.steadyNavierStokes
        elif param.pde_type == 'viscousBurgers':
            make_target = method.steadyViscousBurgers
        elif param.pde_type == 'Stokes':
            make_target = method.Laplace
        elif param.pde_type == 'Laplace':
            make_target = method.Laplace

elif solver_parameters['method']['type'] == 'direct':
    var_train = {'true_fun': param.true_fun}
    make_target = train.Direct_target
    
elif solver.method == 'finitediff':
    pass
    # import finitediff as method
    # var_train = {}

In [8]:
# Interior
optimizer1.zero_grad()
L2loss_interior, Linfloss_interior, resample_interior = train.interior(IntPoints_batch, num, model, make_target, var_train, dev, Domain.volume, 1.0, 0.0, False)
optimizer1.step()

Total_L2loss_interior = L2loss_interior.cpu().numpy()*np.ones(trainingsteps)
Total_Linfloss_interior = Linfloss_interior.cpu().numpy()*np.ones(trainingsteps)

# Walls
optimizer2.zero_grad()
L2loss_wall, Linfloss_wall, resample_wall = train.boundary(BCPoints_batch, num_wall, model, train.Dirichlet_target, dev, wall_measure, 0.0, False)
optimizer2.step()

Total_L2loss_wall = L2loss_wall.cpu().numpy()*np.ones(trainingsteps)
Total_Linfloss_wall = Linfloss_wall.cpu().numpy()*np.ones(trainingsteps)

# Collect Errors
ErrorPoints.location = create.generate_interior_points(num_error, input_dim, input_range, Domain, Domain.inside)
L2error, Linferror = diagnostics.CalculateError(ErrorPoints_batch, num_error, Domain.volume, model, true_fun, dev)

Total_L2error = L2error.cpu().numpy()*np.ones(trainingsteps)
Total_Linferror = Linferror.cpu().numpy()*np.ones(trainingsteps)

print('No errors in first epoch, training will continue')




No errors in first epoch, training will continue


In [9]:
### Continue training

start_time = time.time()

for step in range(trainingsteps-1):
    
    # Interior
    optimizer1.zero_grad()
    L2loss_interior, Linfloss_interior, resample_interior = train.interior(IntPoints_batch, num, model, make_target, var_train, dev, Domain.volume, 1.0, 0.0, False)
    optimizer1.step()

    Total_L2loss_interior[step+1] = L2loss_interior.cpu().numpy()
    Total_Linfloss_interior[step+1] = Linfloss_interior.cpu().numpy()

    # Wall
    optimizer2.zero_grad()
    L2loss_wall, Linfloss_wall, resample_wall = train.boundary(BCPoints_batch, num_wall, model, train.Dirichlet_target, dev, wall_measure, 0.0, False)
    optimizer2.step()

    Total_L2loss_wall[step+1] = L2loss_wall.cpu().numpy()
    Total_Linfloss_wall[step+1] = Linfloss_wall.cpu().numpy()

    # Collect Errors
    ErrorPoints.location = create.generate_interior_points(num_error, input_dim, input_range, Domain, Domain.inside)
    L2error, Linferror = diagnostics.CalculateError(ErrorPoints_batch, num_error, Domain.volume, model, true_fun,dev)

    Total_L2error[step+1] = L2error.cpu().numpy()
    Total_Linferror[step+1] = Linferror.cpu().numpy()

    # Print statements
    if step == 0:
        current_time = time.time()
        print('Approx time: {:.0f} minutes'.format((current_time - start_time)*trainingsteps/60))

    if (step+1) % print_every == 0:
        current_time = time.time()
        print('step = {0}/{1}, {2:2.3f} s/step, time-to-go:{3:2.0f} min'.format(
                step+1, trainingsteps, (current_time - start_time) / (step + 1), 
            (current_time - start_time) / (step + 1) * (trainingsteps - step - 1)/60))

### Final errors
loss_error = {'loss' : {}, 'errors': {}}
loss_error['loss']['L2 interior loss'] = Total_L2loss_interior
loss_error['loss']['Linf interior loss'] = Total_Linfloss_interior

loss_error['loss']['L2 wall loss'] = Total_L2loss_wall
loss_error['loss']['Linf wall loss'] = Total_Linfloss_wall

loss_error['errors']['L2 error'] = Total_L2error
loss_error['errors']['Linf error'] = Total_Linferror

### Save as pickle file

with open('experiments/' + solver_parameters['savemodel'] + '_losserror.pickle', 'wb' ) as handle:
    pickle.dump(loss_error, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('experiments/' + solver_parameters['savemodel'] + '_parameters.pickle', 'wb') as handle:
    pickle.dump(solver_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save model
torch.save(model.state_dict(), "savedmodels/" + solver_parameters['savemodel'] + ".pt")
print('Done')

Approx time: 0 minutes
step = 10/100, 0.204 s/step, time-to-go: 0 min
step = 20/100, 0.204 s/step, time-to-go: 0 min
step = 30/100, 0.202 s/step, time-to-go: 0 min
step = 40/100, 0.202 s/step, time-to-go: 0 min
step = 50/100, 0.203 s/step, time-to-go: 0 min
step = 60/100, 0.204 s/step, time-to-go: 0 min
step = 70/100, 0.205 s/step, time-to-go: 0 min
step = 80/100, 0.205 s/step, time-to-go: 0 min
step = 90/100, 0.205 s/step, time-to-go: 0 min
Done


In [1]:
test_dict = {'a':2, 'b':4}

if test_dict['c']:
    print('Can check for item in dictionary')
else:
    print('Does not work')

KeyError: 'c'